# 📈 Gradient Boosting (XGBoost) — Previsão de LTV do Cliente
**Pós-Graduação BI & Analytics · ML Aplicado a Decisões de Negócio**

---

### 📡 Sinal de Negócio
> O modelo de churn identificou **~600 clientes em risco**. O time comercial quer ligar para todos com o mesmo desconto de 15%.  
> Mas um cliente com LTV de R$ 500 e outro com LTV de R$ 25.000 merecem o mesmo investimento de retenção?  
> Precisamos **prever o valor futuro de cada cliente** para priorizar o esforço onde o retorno é maior.

### 🏗️ O que vamos construir
Um modelo de **regressão** que prevê o LTV dos próximos 12 meses.  
Primeiro comparamos com OLS para mostrar *por que* precisamos de algo mais sofisticado.

---
### Roteiro
1. Importar e explorar
2. Por que não OLS? (distribuição do target)
3. Baseline: Regressão Linear
4. XGBoost: treinar e avaliar
5. Comparativo de métricas
6. SHAP: abrir a caixa-preta
7. Saída acionável: priorização de retenção

In [ ]:
# ── CÉLULA PRONTA: imports ─────────────────────────────────────────
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

plt.rcParams.update({'figure.dpi': 120, 'axes.spines.top': False, 'axes.spines.right': False})
fmt_r = mtick.FuncFormatter(lambda x, _: f'R${x:,.0f}')
print('✅ Bibliotecas carregadas')

In [ ]:
# ── CÉLULA PRONTA: carregar dados ──────────────────────────────────
URL = 'https://raw.githubusercontent.com/rfranceli-doc/analise_ml_mba_eca/main/clientes_ltv.csv'
# df = pd.read_csv('clientes_ltv.csv')  # local
df = pd.read_csv(URL)
print(f'Shape: {df.shape}')
df.head()

## 2. Por que não OLS? Visualizar a distribuição do target

In [ ]:
# ── CÉLULA PRONTA: distribuição do LTV ────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

axes[0].hist(df['ltv_12m'], bins=60, color='#94a3b8', edgecolor='white')
axes[0].set(title='LTV 12m — distribuição bruta', xlabel='R$')
axes[0].xaxis.set_major_formatter(fmt_r)

axes[1].hist(np.log1p(df['ltv_12m']), bins=60, color='#2f6ec8', edgecolor='white')
axes[1].set(title='LTV 12m — escala log\n(muito mais simétrico)', xlabel='log(R$)')

df.boxplot(column='ltv_12m', by='segmento', ax=axes[2],
           boxprops=dict(color='#2f6ec8'), medianprops=dict(color='#c84b2f'))
axes[2].set(title='LTV por Segmento', xlabel='', ylabel='R$')
axes[2].yaxis.set_major_formatter(fmt_r)
plt.suptitle('')
plt.tight_layout()
plt.show()

print(f'Skewness do LTV: {df.ltv_12m.skew():.2f} (> 1 = assimétrico)')
print(f'P95/P50 ratio: {df.ltv_12m.quantile(0.95)/df.ltv_12m.median():.1f}× — outliers de alto valor')
print('\n💡 OLS penaliza erros quadraticamente. Com distribuição assimétrica,')
print('   os outliers de alto valor vão dominar o modelo.')

## 3. Pré-processamento

In [ ]:
# ── CÉLULA PRONTA: encoding e split ───────────────────────────────
df_model = pd.get_dummies(df.drop(columns=['id_cliente']),
                          columns=['canal_principal','segmento','porte'],
                          drop_first=True)

TARGET = 'ltv_12m'
X = df_model.drop(columns=[TARGET])
y = df_model[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

scaler  = StandardScaler()
Xtr_sc  = scaler.fit_transform(X_train)
Xte_sc  = scaler.transform(X_test)

print(f'Treino: {X_train.shape} | Teste: {X_test.shape}')

## 3. Baseline — Regressão Linear (OLS)

In [ ]:
# ── CÉLULA PRONTA: OLS baseline (código completo) ─────────────────
ols = LinearRegression()
ols.fit(Xtr_sc, y_train)
y_pred_ols = ols.predict(Xte_sc)

rmse_ols = np.sqrt(mean_squared_error(y_test, y_pred_ols))
mae_ols  = mean_absolute_error(y_test, y_pred_ols)
r2_ols   = r2_score(y_test, y_pred_ols)

print(f'OLS — RMSE: R$ {rmse_ols:,.0f} | MAE: R$ {mae_ols:,.0f} | R²: {r2_ols:.3f}')

# Resíduos OLS
fig, ax = plt.subplots(figsize=(7, 4))
res_ols = y_test.values - y_pred_ols
ax.scatter(y_pred_ols, res_ols, alpha=0.3, color='#c84b2f', s=8)
ax.axhline(0, color='#1a1a1a', lw=1)
ax.set(title='Resíduos OLS — Heterocedasticidade visível',
       xlabel='Previsto', ylabel='Resíduo')
ax.xaxis.set_major_formatter(fmt_r)
ax.yaxis.set_major_formatter(fmt_r)
plt.tight_layout()
plt.show()
print('\n💡 Erros maiores para clientes de alto valor — viola homocedasticidade do OLS.')

## 4. XGBoost — Treinar e Avaliar

In [ ]:
# ╔══════════════════════════════════════════════════╗
# ║  COMPLETE AQUI — Treinar XGBRegressor           ║
# ╚══════════════════════════════════════════════════╝
# Dica: from xgboost import XGBRegressor
# Parâmetros sugeridos: n_estimators=300, max_depth=4, learning_rate=0.05,
#                       subsample=0.8, colsample_bytree=0.8, random_state=42
# Pergunta: o que learning_rate controla?
# Pergunta: por que subsample < 1 ajuda?

from xgboost import XGBRegressor

xgb = XGBRegressor(
    n_estimators=___,
    max_depth=___,
    learning_rate=___,
    subsample=___,
    colsample_bytree=0.8,
    random_state=42,
    verbosity=0
)
xgb.fit(___, ___)
y_pred_xgb = xgb.predict(___)

print('✅ XGBoost treinado')

In [ ]:
# ╔════════════════════════════════════════════════════════╗
# ║  COMPLETE AQUI — Calcular RMSE, MAE, R² do XGBoost  ║
# ╚════════════════════════════════════════════════════════╝

rmse_xgb = ___
mae_xgb  = ___
r2_xgb   = ___

print('=' * 55)
print(f"{'Modelo':<12} {'RMSE':>14} {'MAE':>14} {'R²':>8}")
print('-' * 55)
print(f"{'OLS':<12} R$ {rmse_ols:>10,.0f} R$ {mae_ols:>10,.0f} {r2_ols:>8.3f}")
print(f"{'XGBoost':<12} R$ {rmse_xgb:>10,.0f} R$ {mae_xgb:>10,.0f} {r2_xgb:>8.3f}")
print('=' * 55)
print(f'\nGanho de R²: +{(r2_xgb - r2_ols):.3f}')
print(f'Redução de MAE: -R$ {mae_ols - mae_xgb:,.0f} por cliente')

In [ ]:
# ── CÉLULA PRONTA: comparação visual de resíduos ──────────────────
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Distribuição do target
axes[0].hist(y_test, bins=50, color='#94a3b8', edgecolor='white')
axes[0].set(title='Distribuição do LTV\n(assimétrica — OLS sofre)', xlabel='R$')
axes[0].xaxis.set_major_formatter(fmt_r)

# Resíduos OLS
axes[1].scatter(y_pred_ols, y_test.values - y_pred_ols,
                alpha=0.25, color='#c84b2f', s=8)
axes[1].axhline(0, color='#1a1a1a', lw=1)
axes[1].set(title=f'Resíduos OLS\nR²={r2_ols:.3f}', xlabel='Previsto', ylabel='Resíduo')
axes[1].xaxis.set_major_formatter(fmt_r)

# Resíduos XGBoost
axes[2].scatter(y_pred_xgb, y_test.values - y_pred_xgb,
                alpha=0.25, color='#2f6ec8', s=8)
axes[2].axhline(0, color='#1a1a1a', lw=1)
axes[2].set(title=f'Resíduos XGBoost\nR²={r2_xgb:.3f}', xlabel='Previsto', ylabel='Resíduo')
axes[2].xaxis.set_major_formatter(fmt_r)

plt.tight_layout()
plt.show()

## 6. SHAP — Abrindo a Caixa-Preta

> SHAP explica **por que** o modelo previu determinado valor para cada cliente.  
> Responde: *quais variáveis empurraram a previsão para cima ou para baixo?*

In [ ]:
# ╔══════════════════════════════════════════════════╗
# ║  COMPLETE AQUI — SHAP summary_plot              ║
# ╚══════════════════════════════════════════════════╝
# Dica: import shap
#       explainer = shap.TreeExplainer(xgb)
#       shap_values = explainer.shap_values(X_test)
#       shap.summary_plot(shap_values, X_test)

import shap

explainer   = shap.TreeExplainer(___)
shap_values = explainer.shap_values(___)

print('Summary Plot — impacto de cada variável no modelo:')
shap.summary_plot(shap_values, X_test, plot_type='beeswarm', show=True)
print('\n💡 Cada ponto é um cliente. Cor = valor da feature. Posição X = impacto no LTV previsto.')

In [ ]:
# ── CÉLULA PRONTA: dependence plots ───────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Relação NPS × LTV (não-linear)
shap.dependence_plot('nps_score', shap_values, X_test, ax=axes[0], show=False)
axes[0].set_title('NPS Score × SHAP\n(efeito acelera abaixo de 5)')
axes[0].axhline(0, color='#888', lw=0.8, linestyle='--')

# Relação recência × LTV
shap.dependence_plot('recencia_dias', shap_values, X_test, ax=axes[1], show=False)
axes[1].set_title('Recência × SHAP\n(clientes inativos têm LTV menor)')
axes[1].axhline(0, color='#888', lw=0.8, linestyle='--')

plt.tight_layout()
plt.show()
print('\n💡 Essas relações não-lineares existem nos dados.')
print('   O OLS usou um coeficiente médio para toda a faixa — perdeu isso.')

## 7. Saída Acionável — Priorização de Retenção

In [ ]:
# ── CÉLULA PRONTA: cruzar churn risk com LTV previsto ─────────────
# Supondo que lista_risco_crm.csv foi gerado no notebook anterior
try:
    lista_crm = pd.read_csv('lista_risco_crm.csv')
    ltv_full  = pd.DataFrame({'id_cliente': df['id_cliente'],
                               'ltv_previsto': xgb.predict(scaler.transform(
                                   pd.get_dummies(df.drop(columns=['id_cliente','ltv_12m']),
                                                  columns=['canal_principal','segmento','porte'],
                                                  drop_first=True)
                               ))})

    prioridade = lista_crm[lista_crm['alerta'] == 1].merge(ltv_full, on='id_cliente')
    prioridade = prioridade.sort_values('ltv_previsto', ascending=False)

    fig, ax = plt.subplots(figsize=(10, 4))
    scatter = ax.scatter(prioridade['prob_churn'],
                         prioridade['ltv_previsto'],
                         c=prioridade['ltv_previsto'],
                         cmap='RdYlGn', s=20, alpha=0.6)
    ax.axvline(0.5, color='#888', lw=1, linestyle='--')
    ax.axhline(prioridade['ltv_previsto'].median(), color='#888', lw=1, linestyle='--')
    ax.set(xlabel='Probabilidade de Churn', ylabel='LTV Previsto (R$)',
           title='Matriz de Priorização — Alto Risco × Alto Valor')
    ax.yaxis.set_major_formatter(fmt_r)
    plt.colorbar(scatter, ax=ax, label='LTV R$')
    plt.tight_layout()
    plt.show()

    print('Quadrante prioritário (prob > 0.5 E LTV > mediana):')
    q = prioridade[(prioridade.prob_churn > 0.5) &
                   (prioridade.ltv_previsto > prioridade.ltv_previsto.median())]
    print(f'  {len(q):,} clientes')
    print(f'  LTV total em risco: R$ {q.ltv_previsto.sum():,.0f}')
    prioridade.to_csv('lista_retencao_priorizada.csv', index=False)
    print('  ✅ lista_retencao_priorizada.csv exportada')

except FileNotFoundError:
    print('⚠️  Execute o notebook de Logit primeiro para gerar lista_risco_crm.csv')